# Part 1

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import io
import requests

In [2]:
c = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(c.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

In [3]:
column_names = ['Postcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
df=df[df['Borough']!='Not assigned']

In [5]:
df = df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [6]:
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

In [7]:
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df.shape

(103, 3)

# Part 2

In [9]:
df = df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [10]:
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

In [11]:
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [12]:
c.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(c, df, on='Postcode')

In [13]:
df = df[['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
df.shape

(103, 5)

# Part 3

In [19]:
pip install folium

  Using cached folium-0.10.1-py2.py3-none-any.whl (91 kB)
  Using cached branca-0.4.0-py3-none-any.whl (25 kB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
import folium

In [24]:
center_lat=df.Latitude.mean()
center_long=df.Longitude.mean()

hood_map = folium.Map(location=[center_lat, center_long], zoom_start=13)
for i, r in df.iterrows():
    folium.CircleMarker(
        [r['Latitude'], r['Longitude']],
        radius=7,
        color='blue',
        popup=r['Neighborhood'],
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.4
    ).add_to(hood_map)
hood_map